In [1]:
import glob
import os
from pydub import AudioSegment

In [2]:
import pandas as pd
import librosa
import librosa.display
from sklearn import preprocessing

In [3]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        flux = get_spectral_features(audio, sample_rate)[2]
#         zero_crossing_rate = get_zero_crossing_rate(audio)
#         spectral_features = get_spectral_features(audio, sample_rate)
        chroma_features = get_chroma_features(audio, sample_rate)
        return [mfccs, flux] + chroma_features
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None

In [4]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder +'/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'flux', 'chroma_features1', 'chroma_features2', 'chroma_features3', 'class_label'])
    return data

In [5]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, სხვადასხვა), mfccsscaled - (40,)
    return mfccs_scaled

In [6]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
    rate_scaled = np.mean(rate.T,axis=0)
#     rate - (1, სხვადასხვა)
    return rate_scaled

In [7]:
def get_spectral_features(audio, sr):
    sp = librosa.feature.spectral_centroid(audio)
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    onset_env.shape = (onset_env.shape[0], 1)
    sp_scaled = np.mean(sp.T,axis=0)
    a_scaled = np.mean(a.T,axis=0)
    onset_env_scaled = np.mean(onset_env,axis=0)
    return [sp_scaled, a_scaled, onset_env_scaled]

In [8]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, სხვადასხვა)
    chroma_stft_scaled = np.mean(chroma_stft.T,axis=0)
    chroma_cens_scaled = np.mean(chroma_cens.T,axis=0)
    chroma_cq_scaled = np.mean(chroma_cq.T,axis=0)
    return [chroma_stft_scaled, chroma_cens_scaled, chroma_cq_scaled]

In [9]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, სხვადასხვა)
    pitches_scaled = np.mean(pitches.T,axis=0)
    return pitches_scaled

In [10]:
data = get_all_features('data')

In [69]:
data = data.sample(frac=1).reset_index(drop=True)
data

,mfccs,flux,chroma_features1,chroma_features2,chroma_features3,class_label
0,"[-521.9288, 34.392395, 0.82456005, 29.736013, ...",[1.3306354],"[0.5062552, 0.44439602, 0.40470782, 0.37494418...","[0.3434948442536817, 0.32525485319052805, 0.31...","[0.72829986, 0.6606441, 0.65967566, 0.62540287...",1
1,"[-387.5964, 67.79973, -16.055218, 42.69492, -9...",[1.5889847],"[0.61257035, 0.5969098, 0.5759922, 0.60611045,...","[0.2765593956224461, 0.2235531990643482, 0.258...","[0.65492004, 0.61187834, 0.6270725, 0.6656931,...",2
2,"[-420.42856, 112.21272, 25.69418, 33.082893, 5...",[1.5603743],"[0.5556021, 0.4577424, 0.40394208, 0.45747662,...","[0.30264462066335984, 0.3120826314943783, 0.41...","[0.61677355, 0.6290657, 0.79495, 0.8219344, 0....",5
3,"[-524.34045, 91.013084, -12.496176, 20.46044, ...",[1.8563066],"[0.42336693, 0.41790974, 0.3877137, 0.34192878...","[0.40768162973538374, 0.3649610190863246, 0.33...","[0.71238923, 0.6882575, 0.6068656, 0.5725098, ...",1
4,"[-484.07718, 119.39732, -7.1748457, 24.873108,...",[1.4618049],"[0.42426097, 0.3942306, 0.38945246, 0.3980457,...","[0.3073958773604165, 0.30601482002445596, 0.29...","[0.5800963, 0.5654143, 0.5535666, 0.54314256, ...",5
...,...,...,...,...,...,...
928,"[-530.86957, 110.248116, 9.260095, 13.359308, ...",[1.1506001],"[0.607899, 0.55921835, 0.48367622, 0.4851535, ...","[0.3616851769708119, 0.36264777244563484, 0.28...","[0.7524747, 0.7312055, 0.64188653, 0.5847164, ...",4
929,"[-507.34, 42.905647, -8.147321, 21.360912, -10...",[1.5070662],"[0.7160474, 0.5838002, 0.53165215, 0.49186018,...","[0.285277751948964, 0.23557629244743383, 0.266...","[0.56895983, 0.53050596, 0.5633019, 0.63630855...",5
930,"[-483.83524, 118.644966, 1.9847218, 14.862812,...",[1.6243234],"[0.26694456, 0.24617197, 0.23898289, 0.2179137...","[0.33420569859550425, 0.2822349605236328, 0.20...","[0.5594194, 0.5144702, 0.44180936, 0.42441007,...",2
931,"[-505.2874, 43.959133, -3.3015234, -5.3747606,...",[1.0517358],"[0.22057894, 0.21237938, 0.2814347, 0.2752888,...","[0.14392476337869445, 0.17119168542965374, 0.2...","[0.37823856, 0.38974655, 0.44068703, 0.4770278...",4


In [70]:
test_data = get_all_features('test')
test_data = test_data.sample(frac=1).reset_index(drop=True)
test_data

,mfccs,flux,chroma_features1,chroma_features2,chroma_features3,class_label
0,"[-444.93436, 50.723774, 18.369139, 16.19961, 7...",[1.4494555],"[0.59621143, 0.494852, 0.43561548, 0.47940192,...","[0.34556402520548585, 0.25663220899678424, 0.2...","[0.7199497, 0.60318553, 0.60958517, 0.5880586,...",3
1,"[-473.3743, 66.0062, 31.40291, 21.405191, 2.69...",[1.5360807],"[0.7186288, 0.74076194, 0.6371092, 0.60031646,...","[0.3600979438466503, 0.35802463179033467, 0.27...","[0.7621373, 0.7379337, 0.6045187, 0.5916448, 0...",5
2,"[-523.36505, 68.824005, 23.7662, 25.582096, 1....",[1.4281317],"[0.5754428, 0.48190477, 0.5054778, 0.5513183, ...","[0.23129685834179017, 0.23201293380356736, 0.2...","[0.563202, 0.54613763, 0.5953624, 0.62446225, ...",5
3,"[-495.3481, 68.46826, 18.524708, 18.044788, 1....",[1.4451244],"[0.6375735, 0.6026951, 0.5888851, 0.5831649, 0...","[0.35582834306033984, 0.2944865334980943, 0.27...","[0.7645002, 0.6601742, 0.567734, 0.5899846, 0....",2
4,"[-342.99542, 101.27446, 11.277179, 30.345806, ...",[1.2384385],"[0.58700377, 0.59619796, 0.64292204, 0.6882647...","[0.3315824026474964, 0.34064613777480524, 0.32...","[0.707168, 0.74272794, 0.6717075, 0.63860255, ...",3
5,"[-478.53287, 65.18405, 17.451687, 21.892368, 7...",[1.5289221],"[0.6736648, 0.6688613, 0.60539407, 0.60239834,...","[0.3421289642681023, 0.33528348089142884, 0.27...","[0.7224362, 0.73755556, 0.62307197, 0.6196807,...",1
6,"[-359.46326, 94.03071, 19.335268, 28.709099, 0...",[1.3419253],"[0.5105883, 0.4716209, 0.4983677, 0.56655645, ...","[0.3650957898682284, 0.32224551534095014, 0.30...","[0.7235571, 0.6515522, 0.6387224, 0.6263556, 0...",5
7,"[-463.0764, 106.60141, 13.516125, 18.425188, 9...",[1.4552801],"[0.76244384, 0.5741109, 0.4730989, 0.4776492, ...","[0.4076168101881672, 0.38264476665599395, 0.27...","[0.7615315, 0.7084449, 0.58817947, 0.49882048,...",2
8,"[-308.6407, 80.72448, -20.594618, 59.448902, -...",[2.643142],"[0.55430716, 0.54594326, 0.47227216, 0.3990189...","[0.26413487880432196, 0.25113287839075993, 0.2...","[0.623919, 0.6156584, 0.63020647, 0.69440633, ...",4
9,"[-475.99872, 97.237366, 26.406332, 29.371502, ...",[1.9520866],"[0.51993966, 0.5636841, 0.56326824, 0.4756088,...","[0.31744241277932217, 0.34654344375363483, 0.3...","[0.70439345, 0.69583005, 0.72975284, 0.6737864...",5


In [71]:
from sklearn.model_selection import train_test_split 
labels = [[i] for i in data['class_label']]
testLabels = [[i] for i in test_data['class_label']]
y_train = np.array(labels)
y_test = np.array(testLabels)

# x_train, x_test, y_train, y_test = train_test_split(data, np.array(labels), test_size=0.1, random_state = 127)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# One-hot encode

In [72]:
from sklearn.preprocessing import OneHotEncoder
def get_one_hot(y):
    encoder = OneHotEncoder(sparse=False)
    y_onehot = encoder.fit_transform(y)
    return y_onehot

In [73]:
y_onehot_train = get_one_hot(y_train)
y_onehot_test = get_one_hot(y_test)
print(y_onehot_train.shape, y_onehot_test.shape)

(933, 5) (25, 5)


/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "ca

# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [74]:
def get_processed_X(x):
    X = x.drop(columns=['class_label'])
    X = X.values
    ls = [] #featurebi gvinda da magitom
    for i in range(X.shape[0]):
        features = []
        for j in range(X.shape[1]):
            for k in X[i][j]:
                features.append(k)
        ls.append(features)
    res = np.array(ls)
#     აქ ჩავამატე
#     print(preprocessing.scale(res))
    return preprocessing.scale(res)

In [75]:
ls_train = get_processed_X(data)
ls_test = get_processed_X(test_data)
print(ls_train.shape, ls_test.shape)

(933, 137) (25, 137)


 # ნეირონული ქსელის კოდი

In [76]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [77]:
import math

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]

In [78]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = []
    for i in range(z3.shape[0]):
        z = [z3[i, j] for j in range(z3.shape[1])]
        h.append(softmax(z))
    h = np.array(h)
#     h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [79]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

#     J = (-1 / m) * np.sum(y * np.log(h).T) #softmax
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [80]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [81]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    J = cost(params,input_size, hidden_size, num_labels, X, y, learning_rate)

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [82]:
# initial setup
input_size = ls_train.shape[1]
hidden_size = 50
num_labels = y_onehot_train.shape[1]
learning_rate = 0.7

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls_train.shape[0]
ls_train = np.matrix(ls_train)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((50, 138), (5, 51))

In [83]:
a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape
print(h)
print(cost(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate))

# softmax(z3[0])
# p = []
# for i in range(h.shape[0]):
#     z = sum([h[i, j] for j in range(h.shape[1])])
#     pla = [h[i, j]/z for j in range(h.shape[1])]
#     p.append(pla)
# p = np.array(p)
# p

[[0.19990302 0.14871108 0.22367562 0.2227085  0.20500177]
 [0.18950997 0.14201142 0.22422605 0.23688456 0.207368  ]
 [0.20488652 0.13972391 0.19842752 0.25114124 0.20582081]
 ...
 [0.19167735 0.17290624 0.22783589 0.18648889 0.22109164]
 [0.19813264 0.17294418 0.2010549  0.19595796 0.23191031]
 [0.21815672 0.15145488 0.19275814 0.21778982 0.21984044]]
2.5359943264068243


In [84]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate)
J, grad.shape

(2.5359943264068243, (7155,))

In [85]:
def get_accuracy(h, y):
    sm = 0
    for a,i in zip(h, y):
        sm+=a[(i[0]-1)]
    sm/= y.shape[0]
    return sm

In [86]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 0.2885099096184886
     jac: array([-1.24826637e-04, -1.68111289e-04, -1.21375064e-04, ...,
       -3.12395985e-04, -1.42112616e-04,  6.45684865e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 18
  status: 3
 success: False
       x: array([ 0.88486136,  0.12525797, -0.23698172, ..., -0.23820561,
       -1.68207844, -0.68616277])

In [87]:
ls_train = np.matrix(ls_train)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
# y_pred_train = np.array(np.argmax(h, axis=1) + 1)
print(h[0])
accuracy = get_accuracy(h, y_train)
print(h.shape, y_train.shape)
print(accuracy)


[9.57143575e-01 1.04170818e-04 1.63302757e-04 6.64850472e-03
 3.59404469e-02]
(933, 5) (933, 1)
0.9480655399791952


In [88]:
ls_test = np.matrix(ls_test)

a1, z2, a2, z3, h = forward_propagate(ls_test, theta1, theta2)
# y_pred = np.array(np.argmax(h, axis=1) + 1)

accuracy = get_accuracy(h, y_test)
print(accuracy)


0.5250951007657602


In [106]:
# correct = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
# accuracy = (sum(map(int, correct)) / float(len(correct)))
# print(accuracy)

In [101]:
# correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
# accuracy = (sum(map(int, correct)) / float(len(correct)))
# print(accuracy)

# ჩაწერა-წაკითხვა

In [127]:
# thetas = []
# thetas.append(theta1)
# thetas.append(theta2)
# np.save('weights', thetas)
# theta1, theta2 = np.load('weights.npy', allow_pickle = True)


# გვჭირდება !pip install XlsxWriter

In [162]:
import xlsxwriter
workbook = xlsxwriter.Workbook("result.xlsx")
worksheet = workbook.add_worksheet()
row = 0 
col = 0

for a, b, c, d, e in h:
    worksheet.write(row, col, a)
    worksheet.write(row, col+1, a)
    worksheet.write(row, col+2, a)
    worksheet.write(row, col+3, a)
    worksheet.write(row, col+4, a)
    row+=1
    
workbook.close()